# Notebook 03: Ingesta y Proyección Socioeconómica (CASEN)

**Objetivo:** Generar la serie temporal completa (2015-2025) de ingresos y pobreza por comuna.

**Corrección 2022:** La variable `region` se extrae desde la base de datos principal, y se cruza con la base geográfica solo para obtener la `comuna`.

**Fuentes:**
1. **2015/2017:** Metadatos internos (.sav).
2. **2020:** Base .sav + Diccionario Excel (Hoja 'HdR').
3. **2022:** Fusión Base Principal (con Región) + Base Geo (Comuna) + Diccionario Excel.

In [1]:
import pandas as pd
import numpy as np
import pyreadstat
import os
import unicodedata

# --- CONFIGURACIÓN ---
RAW_DIR = os.path.join('..', 'data', 'raw')
PROCESSED_DIR = os.path.join('..', 'data', 'processed')
OUTPUT_FILE = os.path.join(PROCESSED_DIR, 'socioeconomico_anual_completo.csv')

# Archivos
FILES = {
    '2015': os.path.join(RAW_DIR, 'Casen 2015.sav'),
    '2017': os.path.join(RAW_DIR, 'Casen 2017.sav'),
    '2020_data': os.path.join(RAW_DIR, 'Base_de_datos_Casen_2020.sav'),
    '2020_codigos': os.path.join(RAW_DIR, 'libro_de_codigos_base_de_datos_casen_2020.xlsx'),
    '2022_main': os.path.join(RAW_DIR, 'Base de datos Casen 2022 SPSS_18 marzo 2024.sav'),
    '2022_geo': os.path.join(RAW_DIR, 'Base de datos provincia y comuna Casen 2022 SPSS.sav'),
    '2022_codigos': os.path.join(RAW_DIR, 'Libro de codigos Base de datos provincia y comuna Casen 2022.xlsx')
}

os.makedirs(PROCESSED_DIR, exist_ok=True)

## 1. Funciones Auxiliares

In [2]:
def normalizar_texto(texto):
    if pd.isna(texto): return ""
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

def calcular_metricas(df, anio, col_comuna='nombre_comuna'):
    # Estandarizar y calcular métricas
    df.rename(columns={c: c.lower() for c in df.columns}, inplace=True)
    col_comuna = col_comuna.lower()
    
    # Pobreza: 1=Extrema, 2=No Extrema -> Ambos son Pobreza
    if 'pobreza' in df.columns:
        df['es_pobre'] = df['pobreza'].isin([1, 2]).astype(int)
    else:
        df['es_pobre'] = 0

    resumen = df.groupby(col_comuna).agg(
        ingreso_promedio=('ytot', 'mean'),
        tasa_pobreza=('es_pobre', 'mean')
    ).reset_index()
    
    resumen.rename(columns={col_comuna: 'nombre_comuna'}, inplace=True)
    resumen['anio'] = int(anio)
    return resumen

## 2. Extractores de Diccionarios

In [3]:
def extraer_diccionario_2020(ruta_excel):
    """Extrae códigos de la hoja 'HdR' buscando el ancla 1101."""
    print(f"  📖 Leyendo códigos 2020 (HdR): {os.path.basename(ruta_excel)}")
    try:
        df = pd.read_excel(ruta_excel, sheet_name='HdR', header=None)
        mask = df.astype(str).apply(lambda x: x.str.contains('^1101$|^1101.0$', regex=True))
        coords = np.argwhere(mask.values)
        
        if len(coords) > 0:
            row_start, col_idx = coords[0]
            data_block = df.iloc[row_start:, [col_idx, col_idx+1]].copy()
            data_block.columns = ['codigo', 'nombre']
            data_block = data_block.dropna()
            data_block['codigo'] = pd.to_numeric(data_block['codigo'], errors='coerce')
            data_block = data_block.dropna(subset=['codigo'])
            diccionario = pd.Series(data_block.nombre.values, index=data_block.codigo.astype(int)).to_dict()
            return diccionario
        return {}
    except Exception as e:
        print(f"    ❌ Error 2020 Excel: {e}"); return {}

def extraer_diccionario_2022(ruta_excel):
    """Extrae códigos 2022. Se busca también el ancla 1101 por seguridad."""
    print(f"  📖 Leyendo códigos 2022: {os.path.basename(ruta_excel)}")
    try:
        # Intentar lectura directa primero (fila 7 aprox)
        df = pd.read_excel(ruta_excel, header=None)
        
        # Búsqueda de ancla 1101 (Iquique) para ser infalible
        mask = df.astype(str).apply(lambda x: x.str.contains('^1101$|^1101.0$', regex=True))
        coords = np.argwhere(mask.values)
        
        if len(coords) > 0:
            row_start, col_idx = coords[0]
            # Asumimos col_idx = codigo, col_idx+1 = nombre
            data = df.iloc[row_start:, [col_idx, col_idx+1]].copy()
            data.columns = ['codigo', 'nombre']
            data = data.dropna()
            data['codigo'] = pd.to_numeric(data['codigo'], errors='coerce')
            data = data.dropna(subset=['codigo'])
            return pd.Series(data.nombre.values, index=data.codigo.astype(int)).to_dict()
        
        # Fallback a lectura fija si no encuentra ancla
        print("    ⚠️ Ancla no encontrada, usando iloc fijo [6:, 3:5]")
        df_clean = df.iloc[6:, [3, 4]].copy()
        df_clean.columns = ['codigo', 'nombre']
        df_clean = df_clean.dropna()
        df_clean['codigo'] = pd.to_numeric(df_clean['codigo'], errors='coerce')
        df_clean.dropna(inplace=True)
        return pd.Series(df_clean.nombre.values, index=df_clean.codigo.astype(int)).to_dict()
        
    except Exception as e: 
        print(f"    ❌ Error 2022 Excel: {e}"); return {}

## 3. Procesamiento por Año

In [4]:
def procesar_historico(anio, ruta):
    # 2015 y 2017
    print(f"--- Procesando {anio} ---")
    try:
        df, meta = pyreadstat.read_sav(ruta, disable_datetime_conversion=True)
        df.columns = [c.lower() for c in df.columns]
        if 'region' in df: df = df[df['region'] == 13].copy()
        
        col_nombre = 'comuna'
        if 'comuna' in meta.variable_value_labels:
            df['nombre_comuna'] = df['comuna'].map(meta.variable_value_labels['comuna'])
            col_nombre = 'nombre_comuna'
            
        return calcular_metricas(df, anio, col_comuna=col_nombre)
    except Exception as e: 
        print(f"Error {anio}: {e}"); return None

def procesar_2020():
    # 2020 (Híbrido)
    print("--- Procesando 2020 ---")
    try:
        df, _ = pyreadstat.read_sav(FILES['2020_data'], usecols=['region', 'comuna', 'ytot', 'pobreza'])
        df.columns = [c.lower() for c in df.columns]
        df = df[df['region'] == 13].copy()
        
        mapa = extraer_diccionario_2020(FILES['2020_codigos'])
        if mapa:
            df['nombre_comuna'] = df['comuna'].map(mapa).fillna(df['comuna'].astype(str))
        else:
            df['nombre_comuna'] = df['comuna'].astype(str)
            
        return calcular_metricas(df, 2020, col_comuna='nombre_comuna')
    except Exception as e:
        print(f"Error 2020: {e}"); return None

def procesar_2022():
    # 2022 (Triple Merge)
    print("--- Procesando 2022 ---")
    try:
        # 1. Cargar Main (Incluye Región, Ingresos, Pobreza)
        print("  Cargando Main...")
        df_main, _ = pyreadstat.read_sav(FILES['2022_main'], usecols=['folio', 'ytot', 'pobreza', 'region'])
        
        # 2. Cargar Geo (Incluye Comuna, Folio)
        print("  Cargando Geo...")
        df_geo, _ = pyreadstat.read_sav(FILES['2022_geo'], usecols=['folio', 'comuna'])
        
        # 3. Merge
        df = pd.merge(df_main, df_geo, on='folio')
        
        # 4. Filtrar RM (usando la región del main)
        df = df[df['region'] == 13].copy()
        
        # 5. Mapear
        mapa = extraer_diccionario_2022(FILES['2022_codigos'])
        if mapa:
            df['nombre_comuna'] = df['comuna'].map(mapa).fillna(df['comuna'].astype(str))
        else:
            df['nombre_comuna'] = df['comuna'].astype(str)
        
        return calcular_metricas(df, 2022)
    except Exception as e:
        print(f"Error 2022: {e}"); return None

## 4. Consolidación e Interpolación Final

In [5]:
dfs = []
dfs.append(procesar_historico(2015, FILES['2015']))
dfs.append(procesar_historico(2017, FILES['2017']))
dfs.append(procesar_2020())
dfs.append(procesar_2022())

# Unir resultados
df_base = pd.concat([d for d in dfs if d is not None], ignore_index=True)
df_base['comuna_norm'] = df_base['nombre_comuna'].apply(normalizar_texto)

# --- Interpolación ---
print("\n>>> Generando serie temporal completa...")
comunas = df_base['comuna_norm'].unique()
anios = range(2015, 2026)
grid = pd.MultiIndex.from_product([comunas, anios], names=['comuna_norm', 'anio']).to_frame(index=False)

# Merge del Grid con los Datos Reales
df_merged = pd.merge(grid, df_base, on=['comuna_norm', 'anio'], how='left')

def rellenar_grupo(g):
    g = g.sort_values('anio')
    g['ingreso_promedio'] = g['ingreso_promedio'].interpolate(method='linear').ffill().bfill()
    g['tasa_pobreza'] = g['tasa_pobreza'].interpolate(method='linear').ffill().bfill()
    return g

# Aplicar interpolación usando la variable correcta
df_final = df_merged.groupby('comuna_norm').apply(rellenar_grupo).reset_index(drop=True)

print(f"Guardando: {OUTPUT_FILE}")
df_final.to_csv(OUTPUT_FILE, index=False)
print("✅ Proceso finalizado.")
print(df_final.sample(10))

--- Procesando 2015 ---
--- Procesando 2017 ---
--- Procesando 2020 ---
  📖 Leyendo códigos 2020 (HdR): libro_de_codigos_base_de_datos_casen_2020.xlsx
--- Procesando 2022 ---
  Cargando Main...
  Cargando Geo...
  📖 Leyendo códigos 2022: Libro de codigos Base de datos provincia y comuna Casen 2022.xlsx

>>> Generando serie temporal completa...
Guardando: ..\data\processed\socioeconomico_anual_completo.csv
✅ Proceso finalizado.
       comuna_norm  anio nombre_comuna  ingreso_promedio  tasa_pobreza
562       vitacura  2016           NaN      1.831720e+06      0.000000
412    puente alto  2020   Puente Alto      2.423730e+05      0.096547
385    providencia  2015   Providencia      1.412648e+06      0.008512
53     cerro navia  2024           NaN      3.012317e+05      0.034448
166     la florida  2016           NaN      4.409886e+05      0.037260
232   lo barnechea  2016           NaN      9.631456e+05      0.035612
457          renca  2021           NaN      2.084882e+05      0.067904
1

C:\Users\corre\AppData\Local\Temp\ipykernel_2348\3780308756.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_merged.groupby('comuna_norm').apply(rellenar_grupo).reset_index(drop=True)
